In [7]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity, Statevector
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

from oft import oft
from tools.classical import find_ideal_heisenberg, trotter_heisenberg_qutip, BitHandler
from boltzmann import lookup_table_boltzmann
from tools.quantum import *
from tools.classical import HamHam

np.random.seed(667)

In [8]:
def energy_state_for_bits(bitstring: str) -> np.ndarray:
    """Convert a bitstring to an energy eigenstate."""
    tensor_list = []
    for bit in bitstring:
        if bit == '0':
            tensor_list.append(qt.basis(2, 0))
        else:
            tensor_list.append(qt.basis(2, 1))
    state = qt.tensor(tensor_list).full()
    return state / np.linalg.norm(state)

def projector_to_energy_eigenspace(bitstring: str) -> np.ndarray:
    """Return the projector to the energy eigenstate corresponding to a bitstring."""
    state = energy_state_for_bits(bitstring)
    return state @ state.conj().T

In [9]:
num_energy_bits = 6
beta = 1
phase_bits = '111100'

if len(phase_bits) != num_energy_bits:
    raise ValueError('This is not the right amount of phase bits')
if phase_bits[0] == '1':
    phase = (int(phase_bits[1:], 2) - 2**(num_energy_bits - 1)) #/ 2**num_energy_bits  # exp(i 2pi phase)
else:
    phase = int(phase_bits[1:], 2) #/ 2**num_energy_bits

energy = 2 * np.pi * phase / 2**num_energy_bits
print(f'Energy: {energy}')

boltzmann_fn = lambda beta, w: np.min([1, np.exp(-beta * w)])
boltzmann_state = np.sqrt(boltzmann_fn(beta, energy)) * qt.basis(2, 0).full() \
    + np.sqrt(1 - boltzmann_fn(beta, energy)) * qt.basis(2, 1).full()
boltzmann_state /= np.linalg.norm(boltzmann_state)

state_we_should_get = qt.tensor([
    qt.Qobj(energy_state_for_bits(phase_bits)),
    qt.Qobj(boltzmann_state)
]).full()
state_we_should_get = state_we_should_get / np.linalg.norm(state_we_should_get)


Energy: -0.39269908169872414


In [10]:
boltzmann_circ = lookup_table_boltzmann(num_energy_bits, beta)

qr_boltz = QuantumRegister(1, name='boltz')
qr_energy = QuantumRegister(num_energy_bits, name='w')
circ = QuantumCircuit(qr_boltz, qr_energy)

circ.initialize(Statevector(energy_state_for_bits(phase_bits)), qr_energy)
circ.compose(boltzmann_circ, [qr_boltz[0], *list(qr_energy)], inplace=True)

circ_state = Statevector(circ).data.reshape(2**(num_energy_bits + 1), 1)

trdist = qt.tracedist(qt.Qobj(circ_state), qt.Qobj(state_we_should_get))
print(f'Trace distance: {trdist}')
fid = qt.fidelity(qt.Qobj(circ_state), qt.Qobj(state_we_should_get))
print(f'Fidelity: {fid}')


Trace distance: 0.0
Fidelity: 1.0000000000000233


### W is doing what it should be doing, i.e. put $\sqrt{\gamma(\omega)}$ onto $|0\rangle$

In [11]:
# def Y_angle(omega: float) -> float:
#     boltzmann_weight = np.exp(-beta * omega)
#     return 2 * np.arcsin(np.sqrt(1 - boltzmann_weight))

# omega = 2 * np.pi * int(phase_bits, 2) / 2**(num_energy_bits)
# boltzmann_angle = Y_angle(omega)

# # Create W_{bitstring}
# W = QuantumCircuit(2, name="W")
# W.x(1)
# W.cry(boltzmann_angle, 1, 0)
# W.x(1)
# print(W)
# circ_state_W = Statevector(W).data.reshape(4, 1)
# full_state_we_expect = qt.tensor([
#     qt.basis(2, 0),
#     qt.Qobj(boltzmann_state)
# ]).full()
# # print(full_state_we_expect.full())
# # print(qt.Qobj(circ_state_W))

# trdist = qt.tracedist(qt.Qobj(circ_state_W), qt.Qobj(full_state_we_expect))
# print(f'Trace distance: {trdist}')
# fid_qt = qt.fidelity(qt.Qobj(circ_state_W), qt.Qobj(full_state_we_expect))
# print(f'Fidelity: {fid_qt}')
